# Define UnionFInd

In [1]:
import random
class UnionFind():
    def __init__(self, n):
        self.parent = list(range(n)) # init parent from 0 to n 
        
    def find(self, x):
        while self.parent[x] != x:
            x = self.parent[x]
        return x
    def union(self, i,j):
        # self.parent[self.find(i)] = self.find(j)
        root_i = self.find(i)
        root_j = self.find(j)
        self.parent[root_i] = root_j

# Define percolison class

In [2]:
import numpy as np 
class Percolation():
    def __init__(self,n):
        # define percolation
        self.percolation = [[0]*n for _ in range(n)]
        self.uf  = UnionFind(n*n + 2)
        # print("parent :",self.uf.parent)
        self.n  = n
        self.virPoint1 = 0
        self.virPoint2 = n*n + 1
        #virtural point is 0 and n+1
        for j in range(n):
            self.uf.union(self.virPoint1 , self.idOf(0,j))
            self.uf.union(self.virPoint2 , self.idOf(n-1,j))
        self.id_to_coord = {}
        for i in range(n):
            for j in range(n):
                id = self.idOf(i,j)
                self.id_to_coord[id] = (i,j)

    def idOf(self, i , j):
        return (self.n * i) + (j +  1) 
        
    def open(self,i,j):
        n  = self.n
        if i> 0 and self.isOpen(i-1,j):
            self.uf.union(self.idOf(i,j),self.idOf(i-1,j))
        if j> 0 and self.isOpen(i,j-1):
            self.uf.union(self.idOf(i,j),self.idOf(i,j-1))
        if j< n-1 and self.isOpen(i,j+1):
            self.uf.union(self.idOf(i,j),self.idOf(i,j+1))
        if i< n-1 and  self.isOpen(i+1,j):
            self.uf.union(self.idOf(i,j),self.idOf(i+1,j))
        self.percolation[i][j] = 1
        
    def isOpen(self,i,j):
        return True if self.percolation[i][j] == 1 else False
    
    def isFull(self,i,j):
        if self.uf.find(self.idOf(i,j)) == self.uf.find(0):
            return True
        else: 
            return False
        
    def numberOfOpenSites(self):
        total_sum = sum(sum(row) for row in self.percolation)
        return int(total_sum)
    def percolates(self):
        n = self.n
        if self.uf.find(self.virPoint1) == self.uf.find(self.virPoint2):
            return True
        else:
            return False 


# Class percolationStats

In [3]:
class PercolationStats():
    def __init__(self,n,trials):
        
        self.trials = trials
        self.results_list = []
        self.mean_value = -1
        self.n = n
        for k in range(trials):
            self.per = Percolation(n)
            # print("k: ", k)
            while self.per.percolates()==False:
                self.open_a_size()
            result = (self.per.numberOfOpenSites()) /(n*n)
            self.results_list.append(result)
                        
    def open_a_size(self):
        n = self.n
        index = random.randint(1 , (n*n))
        i , j = self.per.id_to_coord[index]
        
        while self.per.isOpen(i,j):
            index = random.randint(1 , n*n)
            i , j = self.per.id_to_coord[index]
        self.per.open(i,j)
                                     
    def mean(self):
        return sum(self.results_list) / (len(self.results_list))
    
    def stddev(self):
        mean_value = self.mean()
        variance = sum((x-mean_value)**2 for x in self.results_list) / (self.trials -1)
        std_dev = variance**0.5 
        return std_dev
    
    def confidenceLo(self):
        mean_value = self.mean()
        stddev = self.stddev()
        return mean_value - (1.96*stddev / (self.trials **0.5))
        
    def confidenceHi(self):
        mean_value = self.mean()
        stddev = self.stddev()
        return mean_value + (1.96*stddev / (self.trials **0.5))
    

In [4]:
trial = PercolationStats(100,200)
print("mean : ",trial.mean())
print("stddev: ", trial.stddev())
print(f"95% confidence interval: [{trial.confidenceLo()} {trial.confidenceHi()}]")

mean :  0.5913429999999998
stddev:  0.016490513520836903
95% confidence interval: [0.5890575314285768 0.5936284685714229]
